## S-Mobile: Predicting Customer Churn

* Team-lead GitLab userid: yil012
* Group name: Anaconda
* Team member names: Yinluo Li, Yuquan Zheng, Zezheng Hao, Shangfu Chen

## Setup

Please complete this Jupyter notebook by answering the questions in `s-mobile.pdf` on Canvas (week9/). Create a Notebook and HTML file with all your results and comments and push both the Notebook and HTML file to GitLab when your team is done. All results MUST be reproducible (i.e., the TA and I must be able to recreate the HTML from the Jupyter Notebook file without changes or errors). This means that you should NOT use any python-packages that are not part of the rsm-msba-spark docker container.

This is the fourth group assignment for MGTA 455 and you will be using git and GitLab. If two people edit the same file at the same time you could get what is called a "merge conflict". git will not decide for you who's change to accept so the team-lead will have to determine which edits to use. To avoid merge conflicts, **always** "pull" changes to the repo before you start working on any files. Then, when you are done, save and commit your changes, and then push them to GitLab. Make "pull first" a habit!

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit
from utils import functions

# increase plot resolution
# mpl.rcParams["figure.dpi"] = 150

In [2]:
## load the data - this dataset must NOT be changed
s_mobile = pd.read_pickle("data/s_mobile.pkl")
s_mobile["churn_yes"] = (s_mobile["churn"] == "yes").astype(int)

If you want access to the full 5M row dataset, use the dropbox link below.

https://www.dropbox.com/s/eli6a1q6qisacci/s_mobile_1M.pkl?dl=1

The downside to using the dataset with 1M rows is, of course, that estimation time will increase substantially. I do NOT recommend you use this dataset to select your final model or for tuning hyper parameters. You can, however, use this larger dataset to re-estimate your chosen model and generate profit estimates for the representative sample.

In [3]:
# show dataset description
# rsm.describe(s_mobile)

In [4]:
s_mobile.head()

,customer,churn,changer,changem,revenue,mou,overage,roam,conference,months,...,smartphone,highcreditr,mcycle,car,travel,region,occupation,training,representative,churn_yes
0,U86940794,yes,109,81,62,600,141,0,0,8,...,yes,no,no,no,no,CS,other,0.0,0,1
1,U56788559,no,0,3,54,146,0,0,0,8,...,yes,yes,no,no,no,SE,other,0.0,0,0
2,U47928407,no,0,-9,39,1158,96,0,1,9,...,yes,no,yes,no,yes,NW,professional,NaN,1,0
3,U75794640,no,0,-8,35,560,0,0,0,37,...,yes,yes,yes,no,no,NW,retired,1.0,0,0
4,U41010771,no,-2,-8,100,1888,112,9,0,20,...,yes,no,yes,yes,no,SW,other,NaN,1,0


## I. Building the model

### A. Logistical regression on "churn rate"

In [5]:
# add "cweight" to adjust regression

class_weight_dict = {0: 49, 1: 1}
s_mobile["cweight"] = rsm.ifelse(
    s_mobile.churn == "yes", class_weight_dict[1], class_weight_dict[0]
)

In [6]:
# standardized explanatory variables

Xs = rsm.scale_df(
    s_mobile.loc[:, "changer":"occupation"],
    wt=s_mobile.cweight,
    train=s_mobile.training == 1,
)
Xs["churn_yes"] = s_mobile.churn_yes
Xs.head()

,changer,changem,revenue,mou,overage,roam,conference,months,uniqsubs,custcare,...,eqpdays,refurb,smartphone,highcreditr,mcycle,car,travel,region,occupation,churn_yes
0,1.795996,0.648059,0.059201,0.051482,0.583946,-0.090861,-0.126390,-0.534447,-0.299926,-0.285786,...,-0.272399,no,yes,no,no,no,no,CS,other,1
1,-0.146109,-0.134122,-0.032648,-0.364217,-0.453367,-0.090861,-0.126390,-0.534447,0.275305,-0.285786,...,-0.288119,no,yes,yes,no,no,no,SE,other,0
2,-0.146109,-0.254458,-0.204864,0.562407,0.252889,-0.090861,0.310452,-0.483883,-0.299926,1.891054,...,-0.602518,no,yes,no,yes,no,yes,NW,professional,0
3,-0.146109,-0.244430,-0.250789,0.014857,-0.453367,-0.090861,-0.126390,0.931900,-0.299926,-0.285786,...,0.172251,no,yes,yes,yes,no,no,NW,retired,0
4,-0.181744,-0.244430,0.495484,1.230822,0.370598,0.675463,-0.126390,0.072318,0.850537,-0.056645,...,-0.299347,no,yes,no,yes,yes,no,SW,other,0


In [7]:
# drop "revenue", "conference", "custcare", "car" for statistically insignificant

idvar = "customer"
rvar = "churn_yes"
evar = [
    "changer",
    "changem",
    "mou",
    "overage",
    "roam",
    "months",
    "uniqsubs",
    "retcalls",
    "dropvce",
    "eqpdays",
    "refurb",
    "smartphone",
    "highcreditr",
    "mcycle",
    "travel",
    "region",
    "occupation",
]

In [8]:
form = "churn_yes ~ " + " + ".join(evar)
lr = smf.glm(
    formula=form,
    family=Binomial(link=logit()),
    freq_weights=s_mobile.loc[s_mobile.training == 1, "cweight"],
    data=Xs[s_mobile.training == 1],
).fit(cov_type="HC1")

In [9]:
rsm.or_ci(lr, importance=True, data=s_mobile[s_mobile.training == 1]).sort_values(
    "importance", ascending=False
).reset_index(drop=True)

,index,OR,OR%,2.5%,97.5%,p.values,,dummy,importance,wmean,wstd,min,max
0,occupation[T.retired],0.186,-81.4%,0.168,0.207,< .001,***,True,5.369,0.137,0.344,0.0,1.0
1,highcreditr[T.yes],0.481,-51.9%,0.453,0.510,< .001,***,True,2.081,0.180,0.384,0.0,1.0
2,occupation[T.student],1.900,90.0%,1.788,2.018,< .001,***,True,1.900,0.055,0.227,0.0,1.0
3,eqpdays,1.831,83.1%,1.753,1.912,< .001,***,False,1.831,510.298,222.647,22.0,1982.0
4,overage,1.766,76.6%,1.693,1.842,< .001,***,False,1.766,61.625,67.964,0.0,283.0
5,region[T.SW],0.628,-37.2%,0.596,0.661,< .001,***,True,1.594,0.202,0.401,0.0,1.0
6,region[T.NE],0.645,-35.5%,0.612,0.679,< .001,***,True,1.551,0.199,0.399,0.0,1.0
7,mou,0.650,-35.0%,0.612,0.691,< .001,***,False,1.538,543.775,546.068,0.0,7668.0
8,region[T.SE],0.651,-34.9%,0.618,0.685,< .001,***,True,1.536,0.199,0.399,0.0,1.0
9,region[T.NW],0.661,-33.9%,0.628,0.696,< .001,***,True,1.513,0.199,0.399,0.0,1.0


In [10]:
functions.cal_auc(
    Xs[s_mobile.training == 0], lr.predict(Xs[s_mobile.training == 0]), y="churn_yes"
)

0.6946

### B. linear regression on "revenue"

In [11]:
# drop statistically insignificant variables
evar_2 = [
    "changem",
    "mou",
    "overage",
    "roam",
    "months",
    "dropvce",
    "eqpdays",
    "refurb",
]
form2 = "revenue ~ " + " + ".join(evar_2)
reg = smf.ols(formula=form2, data=s_mobile[s_mobile.training == 1]).fit()

## II. Incentive Strategy 1: New phone coupon (epdays+refurb)

### A. Impact on churn rate

In [12]:
# impact of renew handset on churn probability
s_mobile_rep = s_mobile.query("representative == 1 & eqpdays>=730").copy()
s_mobile_rep["p_newphone"] = lr.predict(
    Xs[(s_mobile.representative == 1) & (s_mobile.eqpdays >= 730)].assign(
        refurb="no", eqpdays=0
    )
)
churn_str_1 = (
    s_mobile_rep.loc[s_mobile.eqpdays >= 730, ["churn_yes", "p_newphone"]]
    .agg(["count", "mean"])
    .round(4)
)
churn_str_1

,churn_yes,p_newphone
count,4886.0000,4886.0000
mean,0.0244,0.0164


### B. Impact on CLV

In [13]:
# revenue before taking incentive strategy

rev_str_1_before = s_mobile.query("representative == 1 & eqpdays>=730")[
    "revenue"
].mean()
churn_rate_str_1_before = np.array([0.0244] * 60)
functions.CLV_cal(rev_str_1_before, churn_rate_str_1_before, intro="Before")

Before taking incentive strategy, the average monthly revenue of a single customer is 42.89, and the expected CLV in 5 years will be 662.5.


In [14]:
# revenue after taking incentive strategy

rev_str_1_after = reg.predict(
    s_mobile[(s_mobile.representative == 1) & (s_mobile.eqpdays >= 730)].assign(
        refurb="no", eqpdays=0
    )
).mean()
churn_rate_str_1_after = np.array([0.0164] * 60)
functions.CLV_cal(rev_str_1_after, churn_rate_str_1_after, intro="After")

After taking incentive strategy, the average monthly revenue of a single customer is 48.33, and the expected CLV in 5 years will be 894.05.


##  III. Incentive Strategy 2: Higher plan promotion (overage+mou)

### A. Impact on churn rate

In [15]:
# impact of promoting higher plan on churn probability
s_mobile_rep = s_mobile.query("representative == 1 & overage/(mou-overage)>0.3").copy()
Xs_2 = Xs.copy()
Xs_2 = Xs_2.assign(mou=Xs_2.overage + Xs_2.mou)
Xs_2 = Xs_2.assign(overage=0)
s_mobile_rep["p_newplan"] = lr.predict(Xs_2[s_mobile.representative == 1])
churn_str_2 = (
    s_mobile_rep.loc[s_mobile_rep.overage > 0, ["churn_yes", "p_newplan"]]
    .agg(["count", "mean"])
    .round(4)
)
churn_str_2

,churn_yes,p_newplan
count,5780.0000,5780.0000
mean,0.0287,0.0178


### B. Impact on CLV

In [16]:
rev_str_2_before = s_mobile.query("representative == 1 & overage/(mou-overage)>0.3")[
    "revenue"
].mean()
churn_rate_str_2_before = np.array([0.0287] * 60)
functions.CLV_cal(rev_str_2_before, churn_rate_str_2_before, intro="Before")

Before taking incentive strategy, the average monthly revenue of a single customer is 54.92, and the expected CLV in 5 years will be 774.75.


In [17]:
# estimate the mean revenue of target coustomers after they take our promotion

s_mobile_rep_2 = s_mobile.query(
    "representative == 1 & overage/(mou-overage)>0.3"
).copy()
s_mobile_rep_2 = s_mobile_rep_2.assign(mou=s_mobile_rep_2.mou)
s_mobile_rep_2 = s_mobile_rep_2.assign(overage=0)
rev_str_2_after = reg.predict(s_mobile_rep_2).mean() * 1.3
churn_rate_str_2_after = np.array([0.0178] * 60)
functions.CLV_cal(rev_str_2_after, churn_rate_str_2_after, intro="After")

After taking incentive strategy, the average monthly revenue of a single customer is 50.29, and the expected CLV in 5 years will be 900.53.


## IV. Case analysis

1.  Develop a model to predict customer churn

In [18]:
rsm.or_ci(lr, importance=True, data=s_mobile[s_mobile.training == 1]).sort_values(
    "importance", ascending=False
).reset_index(drop=True)

,index,OR,OR%,2.5%,97.5%,p.values,,dummy,importance,wmean,wstd,min,max
0,occupation[T.retired],0.186,-81.4%,0.168,0.207,< .001,***,True,5.369,0.137,0.344,0.0,1.0
1,highcreditr[T.yes],0.481,-51.9%,0.453,0.510,< .001,***,True,2.081,0.180,0.384,0.0,1.0
2,occupation[T.student],1.900,90.0%,1.788,2.018,< .001,***,True,1.900,0.055,0.227,0.0,1.0
3,eqpdays,1.831,83.1%,1.753,1.912,< .001,***,False,1.831,510.298,222.647,22.0,1982.0
4,overage,1.766,76.6%,1.693,1.842,< .001,***,False,1.766,61.625,67.964,0.0,283.0
5,region[T.SW],0.628,-37.2%,0.596,0.661,< .001,***,True,1.594,0.202,0.401,0.0,1.0
6,region[T.NE],0.645,-35.5%,0.612,0.679,< .001,***,True,1.551,0.199,0.399,0.0,1.0
7,mou,0.650,-35.0%,0.612,0.691,< .001,***,False,1.538,543.775,546.068,0.0,7668.0
8,region[T.SE],0.651,-34.9%,0.618,0.685,< .001,***,True,1.536,0.199,0.399,0.0,1.0
9,region[T.NW],0.661,-33.9%,0.628,0.696,< .001,***,True,1.513,0.199,0.399,0.0,1.0


2. Use the model output from a logistic regression to understand the main drivers of churn and report on the key factors that predict customer churn and their relative importance

We removed insignificant variables: "revenue", "car", "conference" and "custcare" to build the logistic regression to predict customer churn rate. Output from the logistic regression above shows the results of final selection of variables. As we can see, 
- **occupation(retired)** is the most important variable which is to say that among all kinds of occupation, the retired people have the lowest probability to churn. 
- **highcreditr** shows us if the customer has a good credit, he/she is also less likely to churn.
- **occupation(retired)** tells a contrary story, students are more likely to churn.
- **eqpdays** is the number of days that a customer hold his current headset. The longer he holds, the more likely he will churn.
- **overage** represent the extra paid beyond the plan of a customer. The higher the overage reaches, the more likely a customer will churn.
- **region (SW,NE,SE,NW)** shows that compared with customer located in Central Singapore, customers in other locations trend to stay in service.
- **mou** means that the more time customer used, the less likely he/she will churn.
- **refurb** indicates that a customer with a new phone are less likely to churn.

3. Use insights on churn drivers to develop actions/offers/incentives

Among these predictors, we have come up with three strategies to lower churn rate of s mobile’s customers:
- strategy 1, we would encourage the customers who **did not** change their handsets for **more than two years(say 730 days)** to get a **new phone**.
- strategy 2, we would encourage customers to **upgrade their meal** if their overage usage is **more than 30%** of their original meal.
- strategy 3, the above two strategies are based on the adjustable predictors. However, there are also some predictors that we cannot change such as customer's occupation or region where customers live. Therefore, we also have another recommendation for marketing department of the company to focus promotion on people who are less likely to churn, such as retirement people or people live in SW of Singapore.

4. Quantify the impact of these actions/offers/incentives on the probability of churn

- For Strategy 1, Comparing the churn rate before and after the actions, we can see that: the churn rate decreased from 0.0244 to 0.0164，making a 33% decreasing. 

In [19]:
# impact of renew handset on churn probability
churn_str_1

,churn_yes,p_newphone
count,4886.0000,4886.0000
mean,0.0244,0.0164


- For Strategy 2, Comparing the churn rate before and after the actions, we can see that: the churn rate decreased from 0.0287 to 0.0178，making a 38% decreasing.

In [20]:
# impact of promoting higher plan on churn probability
churn_str_2

,churn_yes,p_newplan
count,5780.0000,5780.0000
mean,0.0287,0.0178


5. Decide which actions/offers/incentives to target to which customers
6. Evaluate the economics: For each action/offer/incentive evaluate the profitability implications using a 5-year (60 month) time window

**For strategy 1:**  
The reason of such proposal is that we found new-phones and refurbishment can lead to a drop in churn rate. We would give discount to customers if they refurbish phones. The initial coupon is set as 100 dollars, but it can be increased according to the final CLV calculation.
<br>
<br>
To estimate the revenue in two situations (get a new-phone and not get a new-phone),we build an leaner model to predict the revenue. As we can see from above code, before the strategy the mean revenue will be around \\$43 and after the strategy the mean revenue will be around \\$48.
<br>
<br>
Finally we need some assumptions for CLV calculations:
<br>
1.We assume the Cost_rate to be 40% according to the financial report of T-mobile.
<br>
2.We assume the life-window to be 5 years - 60 months.
<br>
3.We use monthly discount rate based on the fact that normally annual discount rate is 10%.
<br>
4.We assume the billing method is post_bill.
<br>
<br>
For ClV calculation, before the strategy we estimate the CLV to be \\$663 and after the strategy we estimate the CLV to be \\$894.
We can see that the difference will be $231. So our 100-dollar coupon will be worthy, we can even set more discount for this strategy.

**For strategy 2:**  
In this case, the total usage- 'mou' variable is not changed, but our meal is 1.3 times of the original meal and the overage usage will be estimated to drop to 0.
<br>
<br>
Like strategy 1, we used the linear model to estimate the revenue in two different situations. Our assumption is that the upgraded meal will be 30% more expensive and this effect is excluded from ols model. So we multiply the predicted revenue after strategy 2 by 1.3 to justify such effect. We notice that the predicted revenue actually dropped from \\$55 to \\$50. But we have a lower churn rate to compensate for that.
<br>
<br>
Finally we need some assumptions for CLV calculations:
<br>
1.We assume the Cost_rate to be 40% according to the financial report of T-mobile.
<br>
2.We assume the life-window to be 5 years - 60 months.
<br>
3.We use monthly discount rate based on the fact that normally annual discount rate is 10%.
<br>
4.We assume the billing method is post_bill.
<br>
<br>
For ClV calculation, before the strategy we estimate the CLV to be \\$775 and after the strategy we estimate the CLV to be \\$901.
We can see that the difference will be $126. So even if we would have less expected revenue with a drop in overage usage, the profits gained by less churn rates will make up for that loss.